In [5]:
from pandas import read_csv
from feedgen.feed import FeedGenerator

In [6]:
def define_rss_header():
    """Set basic elements at the tome of xml file
    """
    fg = FeedGenerator()
    fg.id('http://lernfunk.de/media/654321')
    fg.title('Mustafa RSS project')
    fg.author( {'name':'Mustafa','email':'mustafadar338@gmail.com'} )
    fg.link( href='http://example.com', rel='alternate' )
    fg.logo('http://ex.com/logo.jpg')
    fg.subtitle('Mustafa cool feed!')
    fg.link( href='http://larskiesow.de/test.atom', rel='self' )
    fg.language('en')
    return fg


In [7]:
def load_data_from_csv(filename):        
    try:
        # load data using "|" as delimiter
        df = read_csv(filename, delimiter="|")
    except Exception as ex:
        print(f"Could not open {filename}. Reason: {str(ex)}")
        return None
        
    # convert columns to lowercase
    columns = [c.lower() for c in df.columns]
    df.columns = columns
    
    return df

In [8]:
def main():

    fg = define_rss_header()
    
    # Get the ATOM feed as string
    atomfeed = fg.atom_str(pretty=True) 
    
    # Get the RSS feed as string
    rssfeed  = fg.rss_str(pretty=True) 
    
    # Time to load data
    filename = 'quotes.csv'
    data = load_data_from_csv(filename)
    if data is None:
        print("Could not load data from csv file")
        return
    
    # Read each row from dataframe and
    # read value from the right column and set it
    # as a value for the current created xml entry
    for i in data.index:        
        # create XML entry (inside item section)
        fe = fg.add_entry()
        
        fe.id(f'http://lernfunk.de/media/654321/{i}')
        fe.title((
            f"<![CDATA[<h1>{data.at[i, 'source']}</h1>\n<h2>"
            f"{data.at[i, 'category']}</h2>]]>"
            f"({data.at[i, 'dob-dod']})"
        ))
        fe.description(f"<![CDATA[<p><img src=\"{data.at[i, 'wpimg']}\"/></p>]]")
        fe.content(f"<![CDATA[<p>{data.at[i, 'quote']}</p>]]")
        fe.link(href=data.at[i, "wplink"])
        
    # Write the RSS feed to a file    
    fg.rss_file('rss.xml') 

In [9]:
if __name__ == '__main__':
    main()